# Machine Learning Foundation

## Course 3, Part c: Support Vector Machines DEMO


## Introdução

Usaremos o conjunto de dados de qualidade do vinho para esses exercícios. Este conjunto de dados contém várias propriedades químicas do vinho, como acidez, açúcar, pH e álcool. Ele também contém uma métrica de qualidade (3-9, com o mais alto sendo melhor) e uma cor (vermelho ou branco). O nome do arquivo é 'Wine_Quality_Data.csv'.



In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns

## Parte 1: Configuração

* Importar os dados.
* Crie a variável de destino 'y' como uma coluna 1/0 onde 1 significa vermelho.
* Crie um 'pairplot' para o conjunto de dados.
* Crie um gráfico de barras mostrando as correlações entre cada coluna e 'y'
* Escolha o máximo de 2 campos correlacionados (usando o valor absoluto das correlações) e crie 'X'
* Use MinMaxScaler para escalar 'X'. Observe que isso produzirá um np.array. Torne-o um DataFrame novamente e renomeie as colunas apropriadamente.


In [ ]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/Wine_Quality_Data.csv", sep=',')


In [ ]:
y = (data['color'] == 'red').astype(int)
fields = list(data.columns[:-1])  # everything except "color"
correlations = data[fields].corrwith(y)
correlations.sort_values(inplace=True)
correlations

In [ ]:
sns.set_context('talk')
#sns.set_palette(palette)
sns.set_style('white')

In [ ]:
sns.pairplot(data, hue='color')

In [ ]:
ax = correlations.plot(kind='bar')
ax.set(ylim=[-1, 1], ylabel='pearson correlation');

In [ ]:
from sklearn.preprocessing import MinMaxScaler

fields = correlations.map(abs).sort_values().iloc[-2:].index
print(fields)
X = data[fields]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=['%s_scaled' % fld for fld in fields])
print(X.columns)

## Parte 2: Limite de Decisão Linear

Nosso objetivo é examinar o limite de decisão de um classificador LinearSVC nesse conjunto de dados. 


In [ ]:
from sklearn.svm import LinearSVC

LSVC = LinearSVC()
LSVC.fit(X, y)

X_color = X.sample(300, random_state=45)
y_color = y.loc[X_color.index]
y_color = y_color.map(lambda r: 'red' if r == 1 else 'yellow')
ax = plt.axes()
ax.scatter(
    X_color.iloc[:, 0], X_color.iloc[:, 1],
    color=y_color, alpha=1)
# -----------
x_axis, y_axis = np.arange(0, 1.005, .005), np.arange(0, 1.005, .005)
xx, yy = np.meshgrid(x_axis, y_axis)
xx_ravel = xx.ravel()
yy_ravel = yy.ravel()
X_grid = pd.DataFrame([xx_ravel, yy_ravel]).T
y_grid_predictions = LSVC.predict(X_grid)
y_grid_predictions = y_grid_predictions.reshape(xx.shape)
ax.contourf(xx, yy, y_grid_predictions, cmap=plt.cm.autumn_r, alpha=.3)
# -----------
ax.set(
    xlabel=fields[0],
    ylabel=fields[1],
    xlim=[0, 1],
    ylim=[0, 1],
    title='decision boundary for LinearSVC');

## Parte 3: Kernel Gaussiano

Vamos agora ajustar um SVC de kernel Gaussiano e ver como o limite de decisão muda.

* Consolide os trechos de código na Pergunta 2 em uma função que leva em um estimador, 'X' e 'y', e produz o gráfico final com limite de decisão. 

In [ ]:
def plot_decision_boundary(estimator, X, y):
    estimator.fit(X, y)
    X_color = X.sample(300)
    y_color = y.loc[X_color.index]
    y_color = y_color.map(lambda r: 'red' if r == 1 else 'yellow')
    x_axis, y_axis = np.arange(0, 1, .005), np.arange(0, 1, .005)
    xx, yy = np.meshgrid(x_axis, y_axis)
    xx_ravel = xx.ravel()
    yy_ravel = yy.ravel()
    X_grid = pd.DataFrame([xx_ravel, yy_ravel]).T
    y_grid_predictions = estimator.predict(X_grid)
    y_grid_predictions = y_grid_predictions.reshape(xx.shape)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.contourf(xx, yy, y_grid_predictions, cmap=plt.cm.autumn_r, alpha=.3)
    ax.scatter(X_color.iloc[:, 0], X_color.iloc[:, 1], color=y_color, alpha=1)
    ax.set(
        xlabel=fields[0],
        ylabel=fields[1],
        title=str(estimator))

In [ ]:
from sklearn.svm import SVC

gammas = [.5, 1, 2, 10]
for gamma in gammas:
    SVC_Gaussian = SVC(kernel='rbf', gamma=gamma)
    plot_decision_boundary(SVC_Gaussian, X, y)

In [ ]:
Cs = [.1, 1, 10]
for C in Cs:
    SVC_Gaussian = SVC(kernel='rbf', gamma=2, C=C)
    plot_decision_boundary(SVC_Gaussian, X, y)

## Parte 4: Comparando os tempos de execução do kernel

Neste exercício, compararemos os tempos de ajuste entre SVC vs Nystroem com rbf kernel.  
<br><br>
Os Jupyter Notebooks fornecem uma função mágica útil **'%timeit'** que executa uma linha e imprime o tempo que levou para se encaixar. Se digitarmos **'%%timeit'** no início da célula, ele produzirá o tempo de execução.


In [ ]:
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

y = data.color == 'red'
X = data[data.columns[:-1]]

kwargs = {'kernel': 'rbf'}
svc = SVC(**kwargs)
nystroem = Nystroem(**kwargs)
sgd = SGDClassifier()

In [ ]:
%%timeit
svc.fit(X, y)

In [ ]:
%%timeit
X_transformed = nystroem.fit_transform(X)
sgd.fit(X_transformed, y)

In [ ]:
X2 = pd.concat([X]*5)
y2 = pd.concat([y]*5)

print(X2.shape)
print(y2.shape)

In [ ]:
%timeit svc.fit(X2, y2)

In [ ]:
%%timeit
X2_transformed = nystroem.fit_transform(X2)
sgd.fit(X2_transformed, y2)